# Build a Custom OpenAI Chatbot with ML-Driven Prompt Engineering

The code below is designed to run as-is with one exception: **the OpenAI API key must be specified**. Edit the cell below to add your API key between the double quotes.

Then, to execute each code cell, click on it and press `Shift` + `Enter` on your keyboard.

In [1]:
import os
import dotenv
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
load_dotenv(".env")

True

## Step 0: Inspecting Non-Customized Results

Before we perform any prompt engineering, **let's ask the OpenAI model some questions and see how it answers**.

(If you encounter an `AuthenticationError` when running this code, make sure that you have added a valid API key to the cell above and executed it.)

In [5]:
client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],
)

In [6]:
ukraine_prompt = """
Question: "When did Russia invade Ukraine?"
Answer:
"""

response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=ukraine_prompt,
    max_tokens=150,
    temperature=0.7,
)

initial_ukraine_answer = response.choices[0].text.strip()
print(initial_ukraine_answer)

Russia invaded Ukraine in February 2014.


In [7]:
twitter_prompt = """
Question: "Who owns Twitter?"
Answer:
"""

response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=twitter_prompt,
    max_tokens=150,
    temperature=0.7,
)

initial_twitter_answer = response.choices[0].text.strip()
print(initial_twitter_answer)

Twitter Inc. is the company that owns the social media platform Twitter. It was founded by Jack Dorsey, Evan Williams, Biz Stone, and Noah Glass in 2006. As of 2021, Jack Dorsey is the current CEO and largest shareholder of Twitter. Other major shareholders include venture capital firms and institutional investors.


The model is answering this way because the training data ends in 2021. **Our task will be to provide context from 2022 to help the model answer these questions correctly.**

# Step 1: Prepare Dataset

## Loading and Wrangling Data

**The data should be loaded into a pandas `DataFrame` called `df` where each row represents a text sample, and there is only one column, `"text"`, which contains the raw text data.**

In this particular case we are collecting data from [the Wikipedia page for the year 2022](https://en.wikipedia.org/wiki/2022) and performing some data wrangling to get it into the appropriate format. Don't worry too much about the details here, since data wrangling looks different for every dataset!

In [11]:
from dateutil.parser import parse
import pandas as pd
import requests

# Get the Wikipedia page for "2022" since OpenAI's models stop in 2021
resp = requests.get("https://en.wikipedia.org/w/api.php?action=query&prop=extracts&exlimit=1&titles=2022&explaintext=1&formatversion=2&format=json")

# Load page text into a dataframe
df = pd.DataFrame()
df["text"] = resp.json()["query"]["pages"][0]["extract"].split("\n")

# Clean up text to remove empty lines and headings
df = df[(df["text"].str.len() > 0) & (~df["text"].str.startswith("=="))]

# In some cases dates are used as headings instead of being part of the
# text sample; adjust so dated text samples start with dates
prefix = ""
for (i, row) in df.iterrows():
    # If the row already has " - ", it already has the needed date prefix
    if " – " not in row["text"]:
        try:
            # If the row's text is a date, set it as the new prefix
            parse(row["text"])
            prefix = row["text"]
        except ValueError:
            # If the row's text isn't a date, add the prefix
            row["text"] = prefix + " – " + row["text"]
df = df[df["text"].str.contains(" – ")]
df.head()

,text
0,– 2022 (MMXXII) was a common year starting on...
1,– The year began with another wave in the COV...
2,– 2022 was also dominated by wars and armed c...
7,– The ongoing Russian invasion of Ukraine esc...
15,January 1 – France takes over the Presidency ...


## Generating Embeddings

We'll use the `Embedding` tooling from OpenAI [documentation here](https://platform.openai.com/docs/guides/embeddings/embeddings) to create vectors representing each row of our custom dataset.

In order to avoid a `RateLimitError` we'll send our data in batches to the `embeddings.create` function.

In [ ]:
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
batch_size = 100
embeddings = []

for i in range(0, len(df), batch_size):
    batch_texts = df.iloc[i:i+batch_size]["text"].tolist()
    
    response = client.embeddings.create(
        model=EMBEDDING_MODEL_NAME,
        input=batch_texts
    )

    # Each `response.data` element is an object with an "embedding"
    embeddings.extend([item.embedding for item in response.data])

df["embeddings"] = embeddings
df.head()

,text,embeddings
0,– 2022 (MMXXII) was a common year starting on...,"[9.842545114224777e-05, -0.01795295812189579, ..."
1,– The year began with another wave in the COV...,"[-0.00460190512239933, -0.02004355937242508, -..."
2,– 2022 was also dominated by wars and armed c...,"[-0.009761546738445759, -0.015438097529113293,..."
7,– The ongoing Russian invasion of Ukraine esc...,"[-0.014669493772089481, -0.0075006913393735886..."
15,January 1 – France takes over the Presidency ...,"[0.030155904591083527, -0.01053685788065195, -..."


In [ ]:
# Dimensionality of the embeddings: 1536
df.loc[0, "embeddings"].shape

(1536,)

In order to avoid having to run that code again in the future, we'll save the generated embeddings as a CSV file.

In [ ]:
df.to_csv("embeddings.csv")

In [ ]:
! ls

__pycache__
casestudy_rag_wikpedia_2022.ipynb
chatbot_memory_management.ipynb
common.py
data
demo-using-bertviz-to-detect-bias-completed.ipynb
demo_cot_and_triggers.ipynb
distances.csv
embeddings.csv
encoding.ipynb
exercise-1-implement-self-attention-solution.ipynb
exercise-2-create-your-own-gpt-model-solution.ipynb
exercise_rag.ipynb
exercise_rag_step_1.ipynb
exercise_rag_step_2.ipynb
exercise_rag_step_3_4.ipynb
helper.py
hugging-face-tokenizer-properties.ipynb
hugging-face-tokenizer.ipynb
openai_test.ipynb
text-generation.ipynb


If you want to stop the tutorial here and come back, you can reload `df` using this code (again adding your API key) rather than generating the embeddings again:

In [103]:
import numpy as np
import pandas as pd
import ast

df = pd.read_csv("embeddings.csv", index_col=0)
df["embeddings"] = df["embeddings"].apply(ast.literal_eval)

In [104]:
df.head(10)

,text,embeddings
0,– 2022 (MMXXII) was a common year starting on...,"[9.842545114224777e-05, -0.01795295812189579, ..."
1,– The year began with another wave in the COV...,"[-0.00460190512239933, -0.02004355937242508, -..."
2,– 2022 was also dominated by wars and armed c...,"[-0.009761546738445759, -0.015438097529113293,..."
7,– The ongoing Russian invasion of Ukraine esc...,"[-0.014669493772089481, -0.0075006913393735886..."
15,January 1 – France takes over the Presidency ...,"[0.030155904591083527, -0.01053685788065195, -..."
16,January 1 – The Regional Comprehensive Econom...,"[-0.0005575703689828515, -0.02416575513780117,..."
17,January 2 – Abdalla Hamdok resigns as Prime Mi...,"[-0.014902799390256405, 0.0011809427523985505,..."
18,January 4 – The five permanent members of the ...,"[-0.0031260892283171415, -0.012686490081250668..."
19,January 5 – A nationwide state of emergency is...,"[-0.002887411043047905, -0.0061250114813447, -..."
20,"January 6 – The CSTO deploys a ""peacekeeping"" ...","[0.013389947824180126, 0.0030728629790246487, ..."


# Step 2: Create a Function that Finds Related Pieces of Text for a Given Question

What we are implementing here is similar to a search engine or recommendation algorithm. We want to sort all of the rows of our dataset from least relevant to most relevant.

This will use the embeddings that we generated previously in order to compare the vectorized version of our question to the vectorized versions of the rows of the dataset.

In [105]:
from scipy.spatial.distance import cosine as cosine_distance
import ast

In [106]:
# Calculate cosine distance between the first two embeddings
# cosine_distance = 1 - cosine_similarity
cosine_distance(df["embeddings"].loc[0], df["embeddings"].loc[1])

np.float64(0.16654763602511913)

In [107]:
import numpy as np
from scipy.spatial.distance import cdist

def get_embedding(text: str, model="text-embedding-ada-002"):
    response = client.embeddings.create(input=[text], model=model)
    return response.data[0].embedding

def distances_from_embeddings(query_embedding, embeddings, distance_metric="cosine"):
    embeddings = np.stack(embeddings)
    return cdist([query_embedding], embeddings, metric=distance_metric)[0]

In [108]:
def get_rows_sorted_by_relevance(question, df):
    """
    Function that takes in a question string and a dataframe containing
    rows of text and associated embeddings, and returns that dataframe
    sorted from least to most relevant for that question
    """
    
    # Get embeddings for the question text
    question_embeddings = get_embedding(question, model=EMBEDDING_MODEL_NAME)
    
    # Make a copy of the dataframe and add a "distances" column containing
    # the cosine distances between each row's embeddings and the
    # embeddings of the question
    df_copy = df.copy()
    df_copy["distances"] = distances_from_embeddings(
        question_embeddings,
        df_copy["embeddings"].values,
        distance_metric="cosine"
    )
    
    # Sort the copied dataframe by the distances and return it
    # (shorter distance = more relevant so we sort in ascending order)
    df_copy.sort_values("distances", ascending=True, inplace=True)
    return df_copy

Let's test that out for a couple different questions:

In [109]:
df = get_rows_sorted_by_relevance("When did Russia invade Ukraine?", df)
df.head()

,text,embeddings,distances
63,March 2 – Russian invasion of Ukraine: Russia ...,"[0.0007359696901403368, -0.018379738554358482,...",0.109123
93,April 3 – Russian invasion of Ukraine: As Russ...,"[-0.012180210091173649, -0.012394352816045284,...",0.111432
248,November 11 – Russian invasion of Ukraine: Ukr...,"[-0.011985236778855324, -0.013950132764875889,...",0.115485
206,September 21 – Russian invasion of Ukraine: Fo...,"[-0.025452367961406708, -0.022093625739216805,...",0.116849
128,May 16 – Russian invasion of Ukraine: The Sieg...,"[-0.018325934186577797, -0.006523981224745512,...",0.119192


In [110]:
df = get_rows_sorted_by_relevance("Who owns Twitter?", df)
df.head()

,text,embeddings,distances
229,October 28 – Elon Musk completes his $44 billi...,"[-0.010196715593338013, -0.016054941341280937,...",0.172969
111,April 25 – Elon Musk reaches an agreement to a...,"[-0.011515718884766102, -0.014650552533566952,...",0.182215
32,January 24 – The federal government under Scot...,"[-0.008993596769869328, -0.008628109470009804,...",0.246601
247,"November 11 – The cryptocurrency exchange FTX,...","[0.002425569109618664, -0.0252867229282856, -1...",0.262706
259,"November 30 – OpenAI releases ChatGPT, an arti...","[-0.011264772154390812, -0.014390257187187672,...",0.264689


In [111]:
df.to_csv("distances.csv", index=False)

In [112]:
df = pd.read_csv("distances.csv")
df["embeddings"] = df["embeddings"].apply(ast.literal_eval)
df.head()

,text,embeddings,distances
0,October 28 – Elon Musk completes his $44 billi...,"[-0.010196715593338013, -0.016054941341280937,...",0.172969
1,April 25 – Elon Musk reaches an agreement to a...,"[-0.011515718884766102, -0.014650552533566952,...",0.182215
2,January 24 – The federal government under Scot...,"[-0.008993596769869328, -0.008628109470009804,...",0.246601
3,"November 11 – The cryptocurrency exchange FTX,...","[0.002425569109618664, -0.0252867229282856, -1...",0.262706
4,"November 30 – OpenAI releases ChatGPT, an arti...","[-0.011264772154390812, -0.014390257187187672,...",0.264689


In [113]:
type(df["embeddings"].loc[0])

list

# Step 3: Create a Function that Composes a Text Prompt

Building on that sorted list of rows, we're going to select the create a text prompt that provides context to a `completions` model in order to help it answer a question. The outline of the prompt looks like this:

```
Answer the question based on the context below, and if the
question can't be answered based on the context, say "I don't
know"

Context:

{context}

---

Question: {question}
Answer:
```

We want to fit as much of our dataset as possible into the "context" part of the prompt without exceeding the number of tokens allowed by the `completions` model, which is currently 4,000. So we'll loop over the dataset, counting the tokens as we go, and stop when we hit the limit. Then we'll join that list of text data into a single string and add it to the prompt.

In [116]:
# If we want to use the full context size,
# we can use the tiktoken library to count tokens:
# tokens of the question and the context = tokens of the prompt
import tiktoken
tokenizer = tiktoken.get_encoding("cl100k_base")
len(tokenizer.encode("Answer the question based on the context"))

7

In [117]:
df = pd.read_csv("embeddings.csv")
df["embeddings"] = df["embeddings"].apply(ast.literal_eval)

In [118]:
import tiktoken

def create_prompt(question, df, max_token_count):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base")
    
    # Count the number of tokens in the prompt template and question
    prompt_template = """
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

{}

---

Question: {}
Answer:"""
    
    current_token_count = len(tokenizer.encode(prompt_template)) + \
                            len(tokenizer.encode(question))
    
    context = []
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:
        
        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count
        
        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)
    

Now let's test that out! We'll use a `max_token_count` below the actual limit just to keep the output shorter and more readable.

In [119]:
print(create_prompt("When did Russia invade Ukraine?", df, 200))


Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

March 2 – Russian invasion of Ukraine: Russia captures its first large city, the Black Sea port of Kherson, as shelling intensifies across many parts of Ukraine, including civilian areas.

###

April 3 – Russian invasion of Ukraine: As Russia's forces retreat from areas near Kyiv, it is accused by Ukraine of war crimes, amid mounting evidence of indiscriminate civilian killings, including the Bucha massacre.

###

November 11 – Russian invasion of Ukraine: Ukrainian forces recapture Kherson, the only regional capital to be taken by Russia since the start of the war.

---

Question: When did Russia invade Ukraine?
Answer:


In [120]:
print(create_prompt("Who owns Twitter?", df, 100))


Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

October 28 – Elon Musk completes his $44 billion acquisition of Twitter.

###

April 25 – Elon Musk reaches an agreement to acquire the social media network Twitter (which he later rebrands as X) for $44 billion USD, which later closes in October.

---

Question: Who owns Twitter?
Answer:


# Step 4: Create a Function that Answers a Question

Our final step is to send that text prompt to a `Completion` model and parse the model output!

In [124]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"

def answer_question(
    question, df, max_prompt_tokens=1800, max_answer_tokens=150
):
    """
    Given a question, a dataframe containing rows of text, and a maximum
    number of desired tokens in the prompt and response, return the
    answer to the question according to an OpenAI Completion model
    
    If the model produces an error, return an empty string
    """
    prompt = create_prompt(question, df, max_prompt_tokens)

    try:
        response = client.completions.create(
            model=COMPLETION_MODEL_NAME,
            prompt=prompt,
            max_tokens=max_answer_tokens,
            temperature=0.7  # optional: defaults to 1.0
        )
        return response.choices[0].text.strip()
    except Exception as e:
        print(f"Error during completion: {e}")
        return ""

Now that we have all of the code complete, let's test it out!

In [125]:
custom_ukraine_answer = answer_question("When did Russia invade Ukraine?", df)
print(custom_ukraine_answer)

February 24 – missiles strike Kyiv.


In [126]:
custom_twitter_answer = answer_question("Who owns Twitter?", df)
print(custom_twitter_answer)

Elon Musk


Below we compare answers with and without our custom prompt:

In [127]:
print(f"""
When did Russia invade Ukraine?

Original Answer: {initial_ukraine_answer}
Custom Answer:   {custom_ukraine_answer}

Who owns Twitter?
Original Answer: {initial_twitter_answer}
Custom Answer:   {custom_twitter_answer}
""")


When did Russia invade Ukraine?

Original Answer: Russia invaded Ukraine in February 2014.
Custom Answer:   February 24 – missiles strike Kyiv.

Who owns Twitter?
Original Answer: Twitter Inc. is the company that owns the social media platform Twitter. It was founded by Jack Dorsey, Evan Williams, Biz Stone, and Noah Glass in 2006. As of 2021, Jack Dorsey is the current CEO and largest shareholder of Twitter. Other major shareholders include venture capital firms and institutional investors.
Custom Answer:   Elon Musk



## Summary

You just used unsupervised machine learning to perform prompt engineering for custom OpenAI chat responses!

In this example, we provided context from 2022 news headlines to answer questions about current events. Try finding your own dataset for some other custom task!